In [2]:
import numpy as np
from itertools import permutations
# useful additional packages 
from random import sample 
import matplotlib.pyplot as plt
import matplotlib.axes as axes
%matplotlib inline
import numpy as np
import networkx as nx
import itertools
from math import *
import timeit
import random
import time
import decimal
from qiskit import BasicAer
from qiskit.tools.visualization import plot_histogram
from qiskit.optimization.ising import max_cut, tsp
from qiskit.aqua.algorithms import VQE, ExactEigensolver
from qiskit.aqua.components.optimizers import SPSA
from qiskit.aqua.components.variational_forms import RY
from qiskit.aqua import QuantumInstance
from qiskit.optimization.ising.common import sample_most_likely
import seaborn as sns

# setup aqua logging
import logging
from qiskit.aqua import set_qiskit_aqua_logging

In [3]:
G = []
p = [4,5,6,7,8,9,10,11]
q = [i**(2) for i in p ]
m = 0
v = open("Matrices.txt"  , "r")
w = v.read().split()
for i in range (len(w)):
    w[i] = int(float(w[i]))
for i in range (len(q)):
    G.append(np.reshape(w[m:m+q[i]] , (p[i] , p[i])))
    m = m + q[i]
# print('Completed Successfully')

In [4]:
S = ["gr17.tsp.txt" ,"gr21.tsp.txt",  "gr24.tsp.txt" , "dantzig42.tsp.txt" , "hk48.tsp.txt", "gr48.tsp.txt"]
nu = [17,21,24,42,48,48]
# K = []
for i in range(len(nu)):
    g = open(S[i]  , "r")
    f = g.read().split()
    r = np.zeros((nu[i],nu[i]))
    n = []
    for i in range(len(f)):
        f[i] = int(f[i])
        if f[i] != 0:
            n.append(f[i])
    count = 0
    h = 0
    for l in range(0, len(r)):
        h = 0
        if l != h:
            p = l
            while p != h:
                r[p,h] = n[count]
                count = count + 1
                h = h + 1                
    C = r + r.T
    G.append(C)

In [5]:
g = open("brazil58.tsp.txt"  , "r")
f = g.read().split()
r = np.zeros((58,58))
n = []
for i in range(len(f)):
    f[i] = int(f[i])
    if f[i] != 0:
        n.append(f[i])
count = 0
for j in range (len(r)):
    for k in range(j+1,len(r)):
        if j != k:
            r[j,k] = n[count]
            count = count + 1
G.append(r+r.T)

In [6]:
def eva(x,M):
    z=0
    for i in range (len(M)-1):
        z = z + M[x[i],x[i+1]]
    z = z + M[x[-1] , x[0]]
    return z

In [7]:
def SimuAnealing(A, T , alpha , L,tol,m):
    f = np.array(sample(list(np.arange(len(A))),len(A) ))
#     f = np.arange(len(A))
    p = eva(f,A) 
    while T > tol:
        for h in range (L):
            q = np.random.randint(len(A) )
            q1 =  np.random.randint(len(A) )
            q2 = np.random.randint(len(A))

            
            f = f
            g = [i for i in f]
            if T > m :
                #3 opt swap 
#             g = np.array(sample(list(np.arange(len(A))),len(A) ))
                temp1 = g[q]
                g[q]  = g[q1]
                g[q1] = g[q2]
                g[q2] = temp1
            else:
                #2 opt swap
                temp1 = g[q]
                g[q] = g[q1]
                g[q1] = temp1

            x = eva(g,A)
            if p > x:
                p = x
                f = g
            else:
                prob = np.exp(-(x-p)/T)
                if prob > np.random.random():
                    p = x
                    f = g
        T = alpha*T

        # print(p)
    return p 

In [11]:
# for j in range(30):
#     print(SimuAnealing(G[-1], 20 , 0.99 , 20,10**(-3),1))

In [9]:
#Grid Search for SA
def grid(A):
    np.random.seed(20)
    alphas =  np.arange(0.9,1,0.01)
    temps = np.arange(10,50,5)
    markov = np.arange(10,50,5)
    tol = np.arange(10**(-5) , 10**(-3) , 0.0001)
    w = np.arange(1,10,1)
    s = []
    for i in range (len(temps)):
        for j in range(len(markov)):
            for k in range (len(alphas)): 
                for l in range (len(tol)):
                    for x in range(len(w)):
                        
                        T = temps[i]
                        alpha = alphas[k]
                        L = markov[j]
                        tolz= tol[l]
                        m = w[x]
                        s.append((SimuAnealing(A, T , alpha , L , tolz , m) , T, alpha, L,tolz,m))
    return(min(s))

In [ ]:
R = np.zeros((15,6))

In [ ]:
for i in range(len(G)):
    t = grid(G[i])
    R[i,0] = len(G[i])
    R[i,1] = t[0] #temp
    R[i,2] =t[1] #alpha
    R[i,3] =t[2] #markov
    R[i,4] =t[3] #tol
    R[i,5] =t[4] #ratio
    print(grid(G[i]))